# **Import Libraries and modules**

In [0]:
# https://keras.io/
!pip install -q keras
import keras

# New Section

In [0]:
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator

import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Add, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

from keras.datasets import mnist

### Load pre-shuffled MNIST data into train and test sets

In [0]:
(trainX, trainy), (testX, testy) = mnist.load_data()

### reshape dataset to have a single channel

In [138]:
width, height, channels = trainX.shape[1], trainX.shape[2], 1
print(width, height, channels)

28 28 1


In [0]:
trainX = trainX.reshape((trainX.shape[0], width, height, channels))
testX = testX.reshape((testX.shape[0], width, height, channels))


### Convert 1-dimensional class arrays to 10-dimensional class matrices


In [0]:
trainy = np_utils.to_categorical(trainy, 10)
testy = np_utils.to_categorical(testy, 10)

### report pixel means and standard deviations

In [141]:
print('Statistics train=%.3f (%.3f), test=%.3f (%.3f)' % (trainX.mean(), trainX.std(), testX.mean(), testX.std()))

Statistics train=33.318 (78.567), test=33.791 (79.172)


### create generator that centers pixel values

In [142]:
datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
datagen

### calculate the mean on the training dataset

In [143]:
datagen.fit(trainX)
print('Data Generator mean=%.3f, std=%.3f' % (datagen.mean, datagen.std))

Data Generator mean=33.318, std=78.567


### demonstrate effect on a single batch of samples

In [144]:
iterator = datagen.flow(trainX, trainy, batch_size=64)
iterator

### get a batch

In [0]:
batchX, batchy = iterator.next()

### pixel stats in the batch

In [146]:
print(batchX.shape, batchX.mean(), batchX.std())

(64, 28, 28, 1) 0.018264811 1.0211288


### demonstrate effect on entire training dataset

In [147]:
iterator = datagen.flow(trainX, trainy, batch_size=len(trainX), shuffle=False)
iterator

In [0]:
from keras.layers import Activation
from keras.layers import LeakyReLU
from keras.regularizers import l2

model = Sequential()
# Extract edges and gradients features
model.add(Convolution2D(16, (3, 3), kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu', input_shape=(28,28,1))) #26
model.add(BatchNormalization())
model.add(Dropout(0.1))

# Trying to extract more features by increasing channels 
model.add(Convolution2D(16, (3, 3),  kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),activation='relu')) #24
model.add(BatchNormalization())
model.add(Dropout(0.1))

# Trying to extract more features by increasing channels 
model.add(Convolution2D(16, (3, 3), kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu')) #22

# reducing the size of parameters
model.add(MaxPooling2D(pool_size=(2, 2))) #11

# Since we have done  MP above we should try using 1x1 and fetch co dependend features.
model.add(Convolution2D(16, (1, 1), kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu')) #11
model.add(BatchNormalization())
model.add(Dropout(0.1))

# Trying to Increase the channels to fetch parts of object
model.add(Convolution2D(16, (3, 3), kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu')) #9
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Convolution2D(16, (3, 3), kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu')) #7
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Convolution2D(16, (3, 3), kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu')) #5
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Convolution2D(16, (3,3), kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))) #3
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Convolution2D(10, (3,3))) 

model.add(Flatten())
model.add(Activation('softmax'))

In [149]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_82 (Conv2D)           (None, 26, 26, 16)        160       
_________________________________________________________________
batch_normalization_64 (Batc (None, 26, 26, 16)        64        
_________________________________________________________________
dropout_64 (Dropout)         (None, 26, 26, 16)        0         
_________________________________________________________________
conv2d_83 (Conv2D)           (None, 24, 24, 16)        2320      
_________________________________________________________________
batch_normalization_65 (Batc (None, 24, 24, 16)        64        
_________________________________________________________________
dropout_65 (Dropout)         (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_84 (Conv2D)           (None, 22, 22, 16)        2320      
__________

### Train the model

In [150]:
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint

def scheduler(epoch, lr):
  return round(0.003 * 1/(1 + 0.319 * epoch), 10)

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.003), metrics=['accuracy'])

# checkpoint

filepath="weights.best"

reduce_LR = LearningRateScheduler(scheduler, verbose=1)
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

#callbacks_list = [reduce_LR, checkpoint]

model.fit(trainX, trainy, batch_size=128, epochs=40, verbose=1, validation_data=(testX, testy), callbacks=[reduce_LR, checkpoint])

Train on 60000 samples, validate on 10000 samples
Epoch 1/40

Epoch 00001: LearningRateScheduler setting learning rate to 0.003.
60000/60000 [==============================] - 13s 216us/step - loss: 0.6817 - acc: 0.9276 - val_loss: 1.0492 - val_acc: 0.7290

Epoch 00001: val_acc improved from -inf to 0.72900, saving model to weights.best
Epoch 2/40

Epoch 00002: LearningRateScheduler setting learning rate to 0.0022744503.
60000/60000 [==============================] - 8s 128us/step - loss: 0.1973 - acc: 0.9712 - val_loss: 0.1992 - val_acc: 0.9658

Epoch 00002: val_acc improved from 0.72900 to 0.96580, saving model to weights.best
Epoch 3/40

Epoch 00003: LearningRateScheduler setting learning rate to 0.0018315018.
60000/60000 [==============================] - 8s 133us/step - loss: 0.1586 - acc: 0.9760 - val_loss: 0.1518 - val_acc: 0.9740

Epoch 00003: val_acc improved from 0.96580 to 0.97400, saving model to weights.best
Epoch 4/40

Epoch 00004: LearningRateScheduler setting learning r

In [0]:
score = model.evaluate(testX, testy, verbose=0)

In [153]:
print(score)

[0.04103585731089115, 0.9926]


In [0]:
y_pred = model.predict(testX)

In [159]:
print(y_pred[:9])
print(testy[:9])

[[9.70721142e-11 9.77623049e-09 6.18370777e-08 3.67111959e-08
  9.80770951e-11 1.69185344e-09 1.86023994e-13 9.99999881e-01
  7.49161497e-11 3.54653551e-09]
 [1.91506842e-06 5.96634777e-07 9.99994278e-01 4.80320628e-09
  6.19780061e-10 1.55935646e-11 2.43169529e-06 1.07593079e-09
  8.71630732e-07 2.78305747e-11]
 [4.71223203e-08 9.99998689e-01 7.80648577e-08 6.76944834e-10
  6.24172628e-07 7.91779726e-08 2.78313365e-07 1.86276779e-07
  1.13499175e-08 1.24552435e-09]
 [9.99869108e-01 5.34586986e-10 2.35874893e-07 1.65215326e-08
  1.26934259e-08 2.24008716e-07 1.27307067e-04 2.19954788e-09
  1.31306376e-06 1.73719445e-06]
 [1.55943019e-11 7.76717535e-09 2.76180415e-11 6.46316542e-11
  9.99984384e-01 1.69378378e-09 1.24674959e-09 8.58622062e-09
  2.97461722e-09 1.56205169e-05]
 [2.35773339e-08 9.99998689e-01 9.10736944e-08 3.12047249e-10
  5.55940517e-07 2.73868821e-08 6.65716087e-08 5.73763089e-07
  7.02585012e-09 2.04527084e-09]
 [7.58167806e-10 2.01829625e-05 2.21290364e-08 5.64205405e